In [193]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession, DataFrame
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import explode, col
from pyspark.mllib.stat import Statistics
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, StringIndexer, ChiSqSelector
from sklearn.model_selection import train_test_split
%matplotlib inline

# Dataset splits

In [194]:
path = 'E:\\Downloads\\diabetes_dataset\\diabetes_012_health_indicators_BRFSS2015.csv'

In [195]:
online_dataset_path = 'E:\\Downloads\\diabetes_dataset\\online.csv'

In [196]:
offline_dataset_path = 'E:\\Downloads\\diabetes_dataset\\offline.csv'

In [197]:
spark = SparkSession.builder.master("local[*]") \
        .appName('HW_3').getOrCreate()

In [210]:
df = spark.read.csv(path, inferSchema=True, encoding='utf-8', header=True)
df.cache()
df.show(20, 2)

+------------+------+--------+---------+---+------+------+--------------------+------------+------+-------+-----------------+-------------+-----------+-------+--------+--------+--------+---+---+---------+------+
|Diabetes_012|HighBP|HighChol|CholCheck|BMI|Smoker|Stroke|HeartDiseaseorAttack|PhysActivity|Fruits|Veggies|HvyAlcoholConsump|AnyHealthcare|NoDocbcCost|GenHlth|MentHlth|PhysHlth|DiffWalk|Sex|Age|Education|Income|
+------------+------+--------+---------+---+------+------+--------------------+------------+------+-------+-----------------+-------------+-----------+-------+--------+--------+--------+---+---+---------+------+
|           0|     1|       1|        1| 40|     1|     0|                   0|           0|     0|      1|                0|            1|          0|      5|      18|      15|       1|  0|  9|        4|     3|
|           0|     0|       0|        0| 25|     1|     0|                   0|           1|     0|      0|                0|            0|          1| 

In [203]:
x = df.drop('Diabetes_012').toPandas()
y = df.select('Diabetes_012').toPandas()
x_train, x_test, y_train, y_test = train_test_split(x,y,stratify=y,test_size=0.2)
df_offline = pd.concat([x_train, y_train], axis=1).reset_index().drop(['index'], axis=1)
df_offline.to_csv(offline_dataset_path, index=False)

In [204]:
df_online = pd.concat([x_test, y_test], axis=1).reset_index().drop(['index'], axis=1)
df_online.to_csv(online_dataset_path, index=False)
df_online.head(20)

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_012
0,0,1,1,31,1,0,0,1,0,0,...,0,5,30,30,1,1,10,3,1,2
1,0,0,1,27,0,0,0,1,0,1,...,0,2,0,0,0,1,9,6,7,0
2,0,0,1,24,0,0,0,0,0,1,...,0,2,0,0,0,1,8,4,3,0
3,0,1,1,24,1,0,0,1,1,1,...,0,3,10,0,0,1,8,6,4,0
4,0,0,1,26,0,0,0,1,1,1,...,0,2,0,0,0,1,3,6,8,0
5,0,0,0,21,1,0,0,1,1,1,...,0,3,4,0,1,0,13,6,5,0
6,1,0,1,34,0,0,0,0,1,1,...,1,3,10,10,0,0,6,3,3,0
7,0,0,1,42,0,0,0,1,1,1,...,0,3,0,4,0,0,6,6,8,0
8,1,1,1,25,0,0,1,1,1,1,...,0,3,0,0,0,0,13,5,4,1
9,1,0,1,23,1,0,1,1,1,1,...,0,3,0,0,0,0,13,5,2,0


In [205]:
df_online['Diabetes_012'].value_counts().sort_index()

0    42741
1      926
2     7069
Name: Diabetes_012, dtype: int64

In [206]:
df_offline['Diabetes_012'].value_counts().sort_index()

0    170962
1      3705
2     28277
Name: Diabetes_012, dtype: int64